In [ ]:
# Seting up spark spark session
import findspark
findspark.init()
from pyspark import SparkConf
from pyspark.sql import SparkSession


In [ ]:
conf = SparkConf().setMaster('local').set("spark.local.dir","C:/Users/Ashutosh/Desktop/Study/Pyspark")
spark = SparkSession \
       .builder \
       .config(conf=conf) \
       .appName('pysparkling water basic') \
       .getOrCreate()

In [ ]:
import h2o
from h2o.automl import H2OAutoML

import pandas as pd
from sklearn import datasets

from pysparkling import *
from pysparkling.ml import *


In [ ]:
hc = H2OContext.getOrCreate(spark)

In [ ]:
h2o_df = h2o.import_file('winequality.csv', sep =';')

In [ ]:
h2o_df.head(5)

In [ ]:
train_data, test_data = h2o_df.split_frame([0.8])

In [ ]:
y = "label"
x = h2o_df.columns
x.remove(y)
train_data[y] = train_data[y].asfactor()
test_data[y] = test_data[y].asfactor()

In [ ]:
automl_model = H2OAutoML(maxModels=5, 
                         maxRuntimeSecs=  60, 
                         labelCol='quality',
                         convertUnknownCategoricalLevelsToNa= True,
                         seed =1)
automl_model.train(X = x, y=y, training_frame = train_data)
automl_model.predict(test_data)

In [ ]:
modelfile = automl_model.download_mojo(path = "C:/Users/Ashutosh/Desktop/Study/Pyspark", get_genmodel_jar = True)
print("Model saved to " + modelfile)

In [ ]:
mojo_file = 'file:///'+ modelfile
settings= H2OMOJOSettings(convertUnknownCategoricalLevelsToNa= True,
                          convertInvalidNumbersToNa= True,
                          withDetailedPredictionCol= True)
h2o_model = H2OMOJOModel.createFromMojo(mojo_file , settings)

In [ ]:
pred_spark_df = load()
prediction = h2o_model.transform(pred_spark_df)
prediction.show()